# 사람인 채용정보 수집기

In [263]:
import time
import math
import re
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from datetime import datetime
from bs4 import BeautifulSoup

In [264]:
Keyword = input() # 키워드 입력

데이터분석가모집


In [265]:
# 검색 URL
url = 'http://www.saramin.co.kr/zf_user/search/recruit?searchType=search&searchword={}&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&panel_type=&search_optional_item=y&search_done=y&panel_count=y&recruitPage={}&recruitSort=relation&recruitPageCount=40&inner_com_type=&quick_apply=&except_read='
pattern_num = re.compile('[0-9]+') # 채용공고 URL 고유값 추출용 정규표현식
saramin = 'http://www.saramin.co.kr'
iframe = 'http://www.saramin.co.kr/zf_user/jobs/relay/view-detail?rec_idx={}&rec_seq=0' # iframe 구조

In [266]:
start = time.time() # 시작 기록 (타이머 시작)
total = pd.DataFrame() # DataFrame를 받기위한 비빌언덕
PageNo = 9999 # 3번 코드 구동을 위한 초기값
print('{: ^102}'.format('{}로 검색결과 수집').format(Keyword))
for pageNo in range(1, 9999):
    
    # (PageNo/40 + 1)보다 pageNo이 작을때까지 반복 즉, 마지막 페이지까지 반복
    if ((PageNo / 40) + 1) < pageNo: break
    r = requests.get(url.format(Keyword, pageNo)) # 키워드, 페이지 값 동적으로 입력
    bs = BeautifulSoup(r.text, 'html.parser') 
    PageNo = int(bs.find('span', class_ = 'cnt_result').text[2:-1].replace(',', '')) # 검색 페이지 분량 추출
    posts = bs.find('div', id = 'recruit_info_list').find_all('div', class_ = 'item_recruit') # 각 채용 공고 태그
    
    for idx, post in enumerate(posts):
        posts = bs.find('div', id = 'recruit_info_list').find_all('div', class_ = 'item_recruit')
        Title = post.find('h2', class_ = 'job_tit').text.strip() # 제목
        tmp = '2020/' + post.find('span', class_ = 'date').text.replace('~', '')

        # 마감일
        if len(tmp) > 12:
            DeadLine = ''.join(pattern.findall(tmp))
        else:
            DeadLine = post.find('span', class_ = 'date').text.replace('~', '')
        Condition = post.find('div', class_ = 'job_condition').find_all('span')
        Area = Condition[0].text # 소재지
        Category = Condition[1].text # 구분
        Education = Condition[2].text # 학력
        EmploymentType = Condition[3].text # 고용형태
        JobSector = post.find('div', class_ = 'job_sector').find_all('a') # 산업군 추출을 위한 정보
        Field1 = JobSector[0].text # 산업군1
        Field2 = JobSector[1].text # 산업군2
        Company = post.find('div', class_ = 'area_corp').find('a').get('title') # 기업명
        JobPost = post.find('h2', class_ = 'job_tit').find('a').get('href') # 채용공고 URL
        CollectDate = datetime.now().date() # D_day
        try:
            D_day = (datetime.strptime(DeadLine, '%Y%m%d').date() - CollectDate).days
        except:
            if '채용' in DeadLine: # 마감일이 채용시까지인 경우 999로 대체
                D_day = '999'
            elif '내일' in DeadLine: # 마감일 내일까지인 경우
                D_day = int(1)
            else : # # 마감일이 오늘인 경우
                D_day = int(0)
        unique_value = pattern.findall(JobPost)[0] # 채용 공고별 URL 고유값 추출
        url_iframe = iframe.format(unique_value) # 각 채용공고별 iframe URL
        r2 = requests.get(url_iframe)
        bs2 = BeautifulSoup(r2.text, 'html.parser')
        Desc = bs2.text.strip().replace('\n', '').replace('\t', '') # 채용상세
        
        total = total.append(pd.DataFrame({'Title' : [Title],
                                           'Description' : [Desc],
                                           'Deadline' : [DeadLine],
                                           'D_day' : [D_day],
                                           'Area' : [Area],
                                           'Category' : [Category],
                                           'Education' : [Education],
                                           'EmploymentType' : [EmploymentType],
                                           'Field1' : [Field1],
                                           'Field2' : [Field2],
                                           'Company' : [Company],
                                           'JobPost' : [saramin + JobPost],
                                           'CollectDate' : [CollectDate]
                                           }))
        if (idx + 1) % 10 == 0 : print('{: ^108}'.format('{} 개 수집 완료'.format((idx + 1) + 40 * (pageNo - 1))))
    print('{:=^108}'.format('페이지 {} / {}'.format(pageNo, round(PageNo / 40) + 1)))
print('{: ^108}'.format('총 {} 개 수집 완료').format(PageNo))
end = time.time()
minute = math.floor((end - start) / 60) # 분 계산
second = round((end - start) % 60, 2) # 초 계산
print('{: ^108}'.format('실행시간 {}분 {} 초'.format(minute, second)))
print('{: ^108}'.format('완료'))
total = total.reset_index(drop = True)

                                                데이터분석가모집로 검색결과 수집                                                 
                                                 10 개 수집 완료                                                 
                                                 20 개 수집 완료                                                 
                                                 30 개 수집 완료                                                 
                                                 40 개 수집 완료                                                 
=================================================페이지 1 / 6==================================================
                                                 50 개 수집 완료                                                 
                                                 60 개 수집 완료                                                 
                                                 70 개 수집 완료                                                 
             

In [267]:
total

,Title,Description,Deadline,D_day,Area,Category,Education,EmploymentType,Field1,Field2,Company,JobPost,CollectDate
0,데이터 분석가 Data Scientist 경력자 모집,채용공고 상세데이터 분석가 Data Scientist 경력자 모집1998년 04월 ...,20201024,17,서울 송파구,경력4년↑,대졸↑,정규직,SI·시스템통합,소프트웨어개발,네이버시스템(주),http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
1,공공 SI PM/PL/ JAVA 개발자 및 빅데이터 분석가 모집,채용공고 상세공공 SI PM/PL/ JAVA 개발자 및 빅데이터 분석가 모집1998...,채용시,999,서울 송파구,경력무관,학력무관,정규직·계약직,SI·시스템통합,DBMS,네이버시스템(주),http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
2,[미리디] 데이터 분석가 (Data Analyst)를 모집합니다.,채용공고 상세,상시채용,999,서울 구로구,경력 2~5년,대졸↑,정규직,전자상거래,쇼핑몰,(주)미리디,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
3,스마트공장 솔루션 개발 및 데이터 분석가 모집 (정규직&계약직),채용공고 상세스마트공장 솔루션 개발 및 데이터 분석가 모집 (정규직&계약직)2006...,20201018,11,부산 동래구,경력무관,학력무관,정규직·무기계약직,SI·시스템통합,ERP,(주)동연에스엔티,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
4,[어니컴(주)] 빅데이터 분석가/알고리즘 개발자 모집(경력무관),채용공고 상세모집부문 및 상세내용모집부문상세내용빅데이터 분석가(데이터 과학자) 0명...,20201029,22,서울 중구,경력무관,대졸↑,정규직,솔루션업체,Linux,어니컴㈜,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2020 하반기 온라인광고/매체광고/광고영업 경력무관 모집,채용공고 상세,20201031,24,서울 구로구,경력무관,고졸↑,정규직·계약직,광고기획사,광고대행사,일프로,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
204,약물 딥러닝 연구원 모집,채용공고 상세신약개발 딥러닝 연구원 모집모집부문 및 상세내용공통 자격요건ㆍ학력 : ...,20201023,16,서울 강남구,경력무관,대졸↑,정규직,SI·시스템통합,인공지능(AI),바이오에이아이,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
205,[체외진단의료기기]국내/해외영업/사업기획/기구개발/HW/FW/QC,채용공고 상세[회사소개] - 헬스케어 체외진단 의료기기 전문기업[근무지] - 경기 ...,20201009,2,서울 금천구,경력 3~12년,대졸↑,정규직,의료·보건,바이오,에이치알포유(주),http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
206,솔루션 개발 / 솔루션 유지보수 / 고객지원,채용공고 상세솔루션 개발 / 솔루션 유지보수 / 고객지원2001년 04월 07일에 ...,20201128,52,서울 송파구,경력무관,학력무관,정규직,DBMS,소프트웨어개발,(주) 제트코,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07


In [268]:
total.to_csv('./채용 기본정보.csv', encoding = 'utf8', index = False)

In [269]:
df = pd.read_csv('./채용 기본정보.csv')
df.head()

,Title,Description,Deadline,D_day,Area,Category,Education,EmploymentType,Field1,Field2,Company,JobPost,CollectDate
0,데이터 분석가 Data Scientist 경력자 모집,채용공고 상세데이터 분석가 Data Scientist 경력자 모집1998년 04월 ...,20201024,17,서울 송파구,경력4년↑,대졸↑,정규직,SI·시스템통합,소프트웨어개발,네이버시스템(주),http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
1,공공 SI PM/PL/ JAVA 개발자 및 빅데이터 분석가 모집,채용공고 상세공공 SI PM/PL/ JAVA 개발자 및 빅데이터 분석가 모집1998...,채용시,999,서울 송파구,경력무관,학력무관,정규직·계약직,SI·시스템통합,DBMS,네이버시스템(주),http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
2,[미리디] 데이터 분석가 (Data Analyst)를 모집합니다.,채용공고 상세,상시채용,999,서울 구로구,경력 2~5년,대졸↑,정규직,전자상거래,쇼핑몰,(주)미리디,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
3,스마트공장 솔루션 개발 및 데이터 분석가 모집 (정규직&계약직),채용공고 상세스마트공장 솔루션 개발 및 데이터 분석가 모집 (정규직&계약직)2006...,20201018,11,부산 동래구,경력무관,학력무관,정규직·무기계약직,SI·시스템통합,ERP,(주)동연에스엔티,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
4,[어니컴(주)] 빅데이터 분석가/알고리즘 개발자 모집(경력무관),채용공고 상세모집부문 및 상세내용모집부문상세내용빅데이터 분석가(데이터 과학자) 0명...,20201029,22,서울 중구,경력무관,대졸↑,정규직,솔루션업체,Linux,어니컴㈜,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020-10-07
